In [8]:
from flask import Flask, request, jsonify, send_file, Response
import requests
import json
import pandas as pd
import random
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

app = Flask(__name__)

# 감성사전 로드
response_SentiWord = requests.get('https://github.com/HMinjae/heartfelt_project/raw/main/SentiWord_info.json')
SentiWord_info = json.loads(response_SentiWord.content.decode('utf-8-sig'))
sentiword_dic = pd.DataFrame(SentiWord_info)

# 감정 답변 데이터 로드
emotion_response_df = pd.read_csv('https://github.com/HMinjae/heartfelt_project/raw/main/%EA%B0%90%EC%A0%95%EB%8B%B5%EB%B3%80.csv')

# 노래 추천 데이터 로드
song_recommendation_df = pd.read_csv('https://github.com/HMinjae/heartfelt_project/raw/main/%EB%85%B8%EB%9E%98%EC%B6%94%EC%B2%9C.csv')

# 행동 추천 데이터 로드
action_recommendation_df = pd.read_csv('https://github.com/HMinjae/heartfelt_project/raw/main/%ED%96%89%EB%8F%99%EC%B6%94%EC%B2%9C.csv')

image_urls = [
    "https://github.com/HMinjae/heartfelt_project/blob/main/1.jpg?raw=true",
    "https://github.com/HMinjae/heartfelt_project/blob/main/2.jpg?raw=true",
    "https://github.com/HMinjae/heartfelt_project/blob/main/3.jpg?raw=true",
    "https://github.com/HMinjae/heartfelt_project/blob/main/4.jpg?raw=true",
    "https://github.com/HMinjae/heartfelt_project/blob/main/5.jpg?raw=true"
]

def sentiment_score(sentence):
    sentiment = 0  
    for i in range(len(sentiword_dic["word"])):
        if sentiword_dic["word"][i] in sentence: 
            sentiment += int(sentiword_dic["polarity"][i])
    return sentiment

def display_image(score):
    image_urls = [
        "https://github.com/HMinjae/heartfelt_project/blob/main/1.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/2.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/3.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/4.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/5.jpg?raw=true"
    ]

    if score >= 10:
        image_url = image_urls[0]
    elif 5 <= score < 10:
        image_url = image_urls[1]
    elif 0 <= score < 5:
        image_url = image_urls[2]
    elif -5 <= score < 0:
        image_url = image_urls[3]
    else:
        image_url = image_urls[4]

    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    plt.imshow(image)
    plt.axis('off')
    plt.show()
    
@app.route('/analyze_emotion', methods=['POST'])
def analyze_emotion():
    data = request.json
    sentence = data.get('sentence', '')

    if not sentence:
        return jsonify({'error': '문장을 입력해주세요.'}), 400

    score = sentiment_score(sentence)

    if score >= 10:
        emotion_status = "매우 기분 좋음"
        emotion_response = random.choice(emotion_response_df.iloc[:, 0].dropna().tolist())
        song_recommendation = random.choice(song_recommendation_df.iloc[:, 0].dropna().tolist())
        action_recommendation = random.choice(action_recommendation_df.iloc[:, 0].dropna().tolist())
        score = sentiment_score(sentence)
    elif 5 <= score < 10:
        emotion_status = "기분 좋음"
        emotion_response = random.choice(emotion_response_df.iloc[:, 1].dropna().tolist())
        song_recommendation = random.choice(song_recommendation_df.iloc[:, 1].dropna().tolist())
        action_recommendation = random.choice(action_recommendation_df.iloc[:, 0].dropna().tolist())
        score = sentiment_score(sentence)
    elif 0 <= score < 5:
        emotion_status = "보통"
        emotion_response = random.choice(emotion_response_df.iloc[:, 2].dropna().tolist())
        song_recommendation = random.choice(song_recommendation_df.iloc[:, 2].dropna().tolist())
        action_recommendation = random.choice(action_recommendation_df.iloc[:, 0].dropna().tolist())
        score = sentiment_score(sentence)
    elif -5 <= score < 0:
        emotion_status = "기분 안좋음"
        emotion_response = random.choice(emotion_response_df.iloc[:, 3].dropna().tolist())
        song_recommendation = random.choice(song_recommendation_df.iloc[:, 3].dropna().tolist())
        action_recommendation = random.choice(action_recommendation_df.iloc[:, 1].dropna().tolist())
        score = sentiment_score(sentence)
    else:
        emotion_status = "매우 기분 안좋음"
        emotion_response = random.choice(emotion_response_df.iloc[:, 4].dropna().tolist())
        song_recommendation = random.choice(song_recommendation_df.iloc[:, 4].dropna().tolist())
        action_recommendation = random.choice(action_recommendation_df.iloc[:, 1].dropna().tolist())
        score = sentiment_score(sentence)

    return jsonify({
        'emotion_status': emotion_status,
        'emotion_response': emotion_response,
        'song_recommendation': song_recommendation,
        'action_recommendation': action_recommendation,
        'score': score
    })

@app.route('/get_image')
def generate_image():
    score = request.args.get('score', type=int)
    
    image_urls = [
        "https://github.com/HMinjae/heartfelt_project/blob/main/1.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/2.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/3.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/4.jpg?raw=true",
        "https://github.com/HMinjae/heartfelt_project/blob/main/5.jpg?raw=true"
    ]

    if score >= 10:
        image_url = image_urls[0]
    elif 5 <= score < 10:
        image_url = image_urls[1]
    elif 0 <= score < 5:
        image_url = image_urls[2]
    elif -5 <= score < 0:
        image_url = image_urls[3]
    else:
        image_url = image_urls[4]

    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)

    return send_file(img_byte_arr, mimetype='image/png')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/May/2024 17:09:08] "POST /analyze_emotion HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2024 17:09:09] "GET /get_image?score=-11 HTTP/1.1" 200 -
